In [1]:
%load_ext autoreload
%autoreload 2

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_v1_tuning"
schema = dj.schema("nnfabrik_v1_tuning")

from nnfabrik.main import *

from nndichromacy.tables.from_mei import TrainedEnsembleModel

Connecting pawelp@134.2.168.16:3306


## Model Setup

In [3]:
model_fn = 'nndichromacy.models.se_core_full_gauss_readout'
model_config = {'pad_input': False,
   'stack': -1,
   'layers': 4,
   'input_kern': 9,
   'gamma_input': 6.3831,
   'gamma_readout': 0.0076,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
   'n_se_blocks': 0,
   'depth_separable': True,
   'grid_mean_predictor': {'type': 'cortex',
    'input_dimensions': 2,
    'hidden_layers': 0,
    'hidden_features': 30,
    'final_tanh': True},
   'share_features': False,
   'share_grid': False,
   'init_sigma': 0.1,
   'init_mu_range': 0.3,
   'gauss_type': 'full'
}

model_config = {'pad_input': False,
   'stack': -1,
   'layers': 4,
   'input_kern': 9,
   'gamma_input': 6.3831,
   'gamma_readout': 0.0076,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
   'n_se_blocks': 0,
   'depth_separable': True,
   'grid_mean_predictor': None,
   'share_features': False,
   'share_grid': False,
   'init_sigma': 0.1,
   'init_mu_range': 0.3,
   'gauss_type': 'full'}

dataset_fn = 'nndichromacy.datasets.static_loaders'
dataset_config = {'paths':['src/data/ImageNet_29767_9_1_all_V1.zip'],
    'normalize': True, 
    'include_behavior': False, 
    'batch_size': 128, 
    'file_tree': True,
    'areas': ['V1']
}

trainer_fn = 'nndichromacy.training.standart_trainer'
trainer_config = {'max_iter': 100,
    'verbose': False,
    'lr_decay_steps': 4,
    'avg_loss': False,
    'lr_init': 0.009
}

## Input Missing Entries

In [4]:
added_dataset = Dataset().add_entry(dataset_fn, dataset_config, dataset_comment='static22564-2-12 full path', skip_duplicates=True)
added_model = Model().add_entry(model_fn, model_config, model_comment='Best Color Model, no shifter, FullGaussianReadout', skip_duplicates=True)
added_trainer = Trainer().add_entry(trainer_fn, trainer_config, trainer_comment="max_iter: 100", skip_duplicates=True)

/src/nnfabrik/nnfabrik/main.py:173: UserWarning: Corresponding entry found. Skipping...
  warnings.warn("Corresponding entry found. Skipping...")
/src/nnfabrik/nnfabrik/main.py:407: UserWarning: Corresponding entry found. Skipping...
  warnings.warn("Corresponding entry found. Skipping...")


## Extract Necessary Hashes

In [5]:
print(f'{"dataset":<8}', added_dataset)
print(f'{"model":<8}', added_model)
print(f'{"trainer":<8}', added_trainer)

dataset  {'dataset_fn': 'nndichromacy.datasets.static_loaders', 'dataset_hash': '72930dd1be6c229df4be82f74803262c'}
model    {'model_fn': 'nndichromacy.models.se_core_full_gauss_readout', 'model_hash': 'fa56487e5ead382e5ee64b0096c20043'}
trainer  {'trainer_fn': 'nndichromacy.training.standart_trainer', 'trainer_hash': 'b4c0c7c93781ae3aa4b502cc9c925b82'}


## Run this as run.py on the compute server

In [52]:
%%writefile run_scripts/train_model_run.py

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_v1_tuning"
schema = dj.schema("nnfabrik_v1_tuning")

from nndichromacy.tables.from_nnfabrik import TrainedModel

model_training_key = []
for seed in range(1000, 6000):
    model_training_key.append({
        'trainer_hash': 'b4c0c7c93781ae3aa4b502cc9c925b82', # Set trainer hash
        'model_hash': 'fa56487e5ead382e5ee64b0096c20043', # Set model hash
        'dataset_hash': '72930dd1be6c229df4be82f74803262c', # Set dataset hash
        'seed': seed
    })

TrainedModel.populate(model_training_key, reserve_jobs=True, display_progress=True)

Overwriting run_scripts/train_model_run.py


### Check the number of jobs running for training of models and optimisation of gabors/meis

In [9]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


### Create ensembles

In [12]:
model_training_key = []
for seed in range(1000, 6000):
    model_training_key.append({
        'trainer_hash': 'b4c0c7c93781ae3aa4b502cc9c925b82', # Set trainer hash
        'model_hash': 'fa56487e5ead382e5ee64b0096c20043', # Set model hash
        'dataset_hash': '72930dd1be6c229df4be82f74803262c', # Set dataset hash
        'seed': seed
    })

In [13]:
TrainedModel() & model_training_key

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nndichromacy.models.se_core_full_gauss_readout,fa56487e5ead382e5ee64b0096c20043,nndichromacy.datasets.static_loaders,72930dd1be6c229df4be82f74803262c,nndichromacy.training.standart_trainer,b4c0c7c93781ae3aa4b502cc9c925b82,1000,"max_iter: 100.Best Color Model, no shifter, FullGaussianReadout.static22564-2-12 full path",0.343055,=BLOB=,pawelp,2021-01-27 11:47:53
nndichromacy.models.se_core_full_gauss_readout,fa56487e5ead382e5ee64b0096c20043,nndichromacy.datasets.static_loaders,72930dd1be6c229df4be82f74803262c,nndichromacy.training.standart_trainer,b4c0c7c93781ae3aa4b502cc9c925b82,2000,"max_iter: 100.Best Color Model, no shifter, FullGaussianReadout.static22564-2-12 full path",0.33962,=BLOB=,pawelp,2021-01-27 11:47:34
nndichromacy.models.se_core_full_gauss_readout,fa56487e5ead382e5ee64b0096c20043,nndichromacy.datasets.static_loaders,72930dd1be6c229df4be82f74803262c,nndichromacy.training.standart_trainer,b4c0c7c93781ae3aa4b502cc9c925b82,3000,"max_iter: 100.Best Color Model, no shifter, FullGaussianReadout.static22564-2-12 full path",0.34299,=BLOB=,pawelp,2021-01-27 11:48:25
nndichromacy.models.se_core_full_gauss_readout,fa56487e5ead382e5ee64b0096c20043,nndichromacy.datasets.static_loaders,72930dd1be6c229df4be82f74803262c,nndichromacy.training.standart_trainer,b4c0c7c93781ae3aa4b502cc9c925b82,4000,"max_iter: 100.Best Color Model, no shifter, FullGaussianReadout.static22564-2-12 full path",0.340788,=BLOB=,pawelp,2021-01-27 11:48:16
nndichromacy.models.se_core_full_gauss_readout,fa56487e5ead382e5ee64b0096c20043,nndichromacy.datasets.static_loaders,72930dd1be6c229df4be82f74803262c,nndichromacy.training.standart_trainer,b4c0c7c93781ae3aa4b502cc9c925b82,5000,"max_iter: 100.Best Color Model, no shifter, FullGaussianReadout.static22564-2-12 full path",0.344268,=BLOB=,pawelp,2021-01-27 11:48:11


In [14]:
TrainedEnsembleModel().create_ensemble(model_training_key, comment='closed-loop mouse ensemble 2')

In [15]:
(TrainedEnsembleModel.Member & model_training_key).fetch('ensemble_hash', 'seed', as_dict=True)

[{'ensemble_hash': '8a2aaa598935fa40a3f7db6f84209d5f', 'seed': 1000},
 {'ensemble_hash': '8a2aaa598935fa40a3f7db6f84209d5f', 'seed': 2000},
 {'ensemble_hash': '8a2aaa598935fa40a3f7db6f84209d5f', 'seed': 3000},
 {'ensemble_hash': '8a2aaa598935fa40a3f7db6f84209d5f', 'seed': 4000},
 {'ensemble_hash': '8a2aaa598935fa40a3f7db6f84209d5f', 'seed': 5000}]